In [5]:
!pip install tensorflow-gpu

     |████████████████████████████████| 394.7MB 42kB/s 


In [6]:
!pip install pandas-datareader

In [2]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [3]:
tf.__version__

'2.4.0'

In [4]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []  ## list of all the brought stock 
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0  ## to choose random action on start of training   
    self.epsilon_final = 0.01  # after model start training choose more appropriate action ,0.01 because even model is fully trained then to choose random action for evn exploration  
    self.epsilon_decay = 0.995 # to decay self.epsilon step by step 
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      
      target = self.model.predict(state)
      target[0][action] = reward
      self.model.fit(state, target, epochs=1, verbose=0)
    
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [5]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [6]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [7]:
dataset11 = data_reader.DataReader('AAPL', data_source="yahoo")

In [8]:
dataset11[:1]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-11,24.764999,24.334999,24.7425,24.6325,198957600.0,22.821304


In [9]:
dataset11.index[0]

Timestamp('2016-01-11 00:00:00')

In [10]:
dataset11.index[-1]

Timestamp('2021-01-06 00:00:00')

In [11]:
def state_creator(data, timestep, window_size):##window_size:- how many previous days need to predict current 
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [12]:
##state will be [40,46,43,42,43.5]
## target will be [47.9]

In [13]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [14]:
window_size = 10  ## 10 previous days to predict new one
episodes = 500   ## how many time to run for whole data set  

batch_size = 32
data_samples = len(data) - 1

In [15]:
trader = AI_Trader(window_size)

In [16]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [17]:
data[:2]

Date
2016-01-11    24.6325
2016-01-12    24.9900
Name: Close, dtype: float64

In [18]:
state_creator(data, 100, 11)

array([[0.56340705, 0.57505349, 0.59085483, 0.60587374, 0.54921519,
        0.49624975, 0.46941337, 0.41338233, 0.45388159, 0.51249721]])

In [19]:
trader.trade([[0.56340705, 0.57505349, 0.59085483, 0.60587374, 0.54921519,
        0.49624975, 0.46941337, 0.41338233, 0.45388159, 0.51249721]])

2

In [20]:
trader.model.predict([[0.56340705, 0.57505349, 0.59085483, 0.60587374, 0.54921519,
        0.49624975, 0.46941337, 0.41338233, 0.45388159, 0.51249721]])

array([[ 0.19520365, -0.0324987 ,  0.09842066]], dtype=float32)

In [21]:
np.argmax([-0.05602242, 0.11945106,  0.10107578])

1

In [ ]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1256 [00:00<?, ?it/s]

Episode: 1/500
AI Trader bought:  $ 24.632500
AI Trader sold:  $ 24.879999  Profit: $ 0.247499
AI Trader bought:  $ 24.282499
AI Trader bought:  $ 24.165001
AI Trader sold:  $ 24.197500  Profit: - $ 0.084999
AI Trader bought:  $ 24.075001
AI Trader sold:  $ 25.355000  Profit: $ 1.189999
AI Trader sold:  $ 24.860001  Profit: $ 0.785000
AI Trader bought:  $ 24.997499
AI Trader bought:  $ 23.522499
AI Trader bought:  $ 24.334999
AI Trader bought:  $ 24.107500
AI Trader bought:  $ 23.620001
AI Trader bought:  $ 24.150000
AI Trader bought:  $ 23.504999
AI Trader sold:  $ 23.752501  Profit: - $ 1.244999
AI Trader bought:  $ 23.747499
AI Trader sold:  $ 23.567499  Profit: $ 0.045000
AI Trader sold:  $ 23.424999  Profit: - $ 0.910000
AI Trader bought:  $ 24.160000
AI Trader sold:  $ 24.530001  Profit: $ 0.422501
AI Trader bought:  $ 24.065001
AI Trader bought:  $ 24.219999
AI Trader bought:  $ 23.672501
AI Trader bought:  $ 24.025000
AI Trader sold:  $ 24.190001  Profit: $ 0.570000
AI Trader b

  3%|▎         | 34/1256 [00:06<19:43,  1.03it/s]

AI Trader bought:  $ 25.132500


  3%|▎         | 35/1256 [00:09<32:14,  1.58s/it]

...

AI Trader bought:  $ 36.382500


 30%|███       | 378/1256 [16:32<42:47,  2.92s/it]

AI Trader sold:  $ 36.435001  Profit: - $ 0.149998


 30%|███       | 379/1256 [16:34<42:06,  2.88s/it]

AI Trader sold:  $ 36.942501  Profit: $ 0.690002
